In [230]:
import numpy as np
import project
import scale

From page 426 in the Bertsikas textbook there is an algorithm that we are testing here 

In [283]:
# need to solve for p and d 
# p are the costs for the objective function of the dual problem 
# d are the directions that we will move in. They are expected to be small, I think.  
# They are the directions that minimize the Taylor series expansion of the Barrier function

# need the matrix A and b for Ax=b

# need to do a horizontal concatenation here to make one augmented matrix.
# aug_A_1 = np.hstack((np.asmatrix(mu*vfunc(X)), A.T))
# aug_b_1 = np.asmatrix(mu*vfunc_1(X))*np.asmatrix(e).T - c

# modularize this

def get_A_b(mu, X, A, e, c):
    aug_A_1 = np.hstack((np.asmatrix(mu*vfunc(X)), A.T))
    aug_b_1 = np.asmatrix(mu*vfunc_1(X))*np.asmatrix(e).T - c
    return aug_A_1, aug_b_1

# get second part 

def get_A_b_2(A):
    aug_A_2 = np.hstack((np.asmatrix(A), np.zeros((A.shape[0],A.shape[0]))))
    aug_b_2 = np.asmatrix(np.zeros((A.shape[0],1)))
    return aug_A_2, aug_b_2

In [284]:
# def generate_A(n):
#     A = -100*np.random.rand(n,n)
#     return A

In [329]:
# initial values 

x_0 = np.matrix([[0.5],[0.5]])
s_0 = np.matrix([[0.5],[0.5]])
p_0 = np.matrix([[0],[1]])

In [403]:
A = np.matrix([[1,1],
               [1,0]])
x = x_0
s = s_0
p = p_0
epsilon = 10**-3
k = 0
alpha = 0.9
mu = 100000
c = np.matrix([[1],[2]])

# cannot divide by 0.0, so need to check for that.  This is slightly inaccurate because if x 
#...is close to zero and negative then it will not preserve the negative (i.e. it puts it to 10**6)

udf = lambda x: (x)**-2 if x!=0.0 else 10**6
vfunc = np.vectorize(udf)

udf_1 = lambda x: (x)**-1 if x!=0.0 else 10**3
vfunc_1 = np.vectorize(udf_1)

# collect the x_s
x_s = []
d_s = []

while s.T*x > epsilon and k < 100:
    
    X = np.diag(x.T.tolist()[0])
    mu = alpha*mu

    aug_A_1, aug_b_1 = get_A_b(mu, X, A, e, c)
    aug_A_2, aug_b_2 = get_A_b_2(A)

    # stack the A's
    A_solv = np.vstack((aug_A_1, aug_A_2))

    # stack the b's
    b_solv = np.vstack((aug_b_1, aug_b_2))
    
    # we are solving these equations in order to get both the directions for where we should go, 
    #....and how far we should move.
    d_p = np.linalg.solve(A_solv, b_solv)
    # extract d and p
    # d is the first "n" elements
    d = d_p[:A.shape[1]]
    # p is the last "m" elements
    p = d_p[-A.shape[0]:]    # why is p so large?

    # update the solutions and change x by "d"
    x_s = x_s + [x]
    d_s = d_s + [d]
    
    x = x + d
    p = p
    # this is making s negative 
    s = c - A.T*p
    
    k += 1
    
print 'done, and s.T*x is ', s.T*x, 'and k is ', k

SyntaxError: invalid syntax (<ipython-input-403-2cea711031b9>, line 40)

### implement Theorem 9.7 on page 426 to get primal and dual solutions

In [373]:
# initial values 

x_0 = np.matrix([[1.0],[1.0]])
s_0 = np.matrix([[1.0],[1.0]])
p_0 = np.matrix([[0],[1]])

S_0 = np.diag(s_0.T.tolist()[0])
X_0 = np.diag(x_0.T.tolist()[0])

e_mat = np.matrix([[i] for i in e])
A = np.matrix([[1,1],
               [1,0]])

How is it possible that mu can be *too* large?  when I set mu=900 the inequality doesn't hold, but I thought that initially it is necessary to set mu to be very high?

In [402]:
mu=3.4
n = A.shape[1]
Beta = 1.0
alpha = 1-((np.sqrt(Beta) - Beta)/(np.sqrt(Beta)+np.sqrt(n)))

# pick initial solutions that satisfy this:

e_mat = np.matrix([[i] for i in e])
L = (1/mu)*np.asmatrix(X_0)*np.asmatrix(S_0)*e_mat - e_mat

LHS = np.linalg.norm(L, ord=2)
RHS = Beta

if LHS <= RHS:
    print 'works LHS is ', LHS, ' which is less than RHS: ', RHS
else:
    print 'does not work. LHS is ', LHS, ' which is greater than RHS: ', RHS

works LHS is  0.998268396969  which is less than RHS:  1.0
